# Lockdown baking - part 2

A project for scraping and analysing data from reddit (r/Sourdough) to explore baking trends during 2020.

Part 2: Data cleaning and classification

# Setup

In [1]:
## set root folder
import os 
os.chdir("..")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Clean data

## Load data

In [3]:
#load scraped datafiles
df = pd.read_csv(".\\data\scraped\post_df_2021-01-07.csv")

In [4]:
# explore shape and columns
print(df.shape)
print(df.columns)
print("")
print(df.head(3))

(41309, 8)
Index(['Unnamed: 0', 'id', 'score', 'created_utc', 'title', 'num_comments',
       'can_mod_post', 'author'],
      dtype='object')

   Unnamed: 0      id  score  created_utc  \
0           0  eibhvl      1   1577839131   
1           1  eibvur      1   1577841129   
2           2  eiby7m      1   1577841483   

                                               title  num_comments  \
0  First attempt at a starter, really hope I mana...             5   
1  Skillet &amp; Dutch Oven Sourdough in the rain...             0   
2  My last bread of 2019. I used Brad and Claire’...             0   

   can_mod_post         author  
0         False  coentertainer  
1         False     Richness69  
2         False     canioli019  


## Remove unnecessary data

In [4]:
print(df["can_mod_post"].value_counts()) 

#since there are no moderators, we don't need this variable
df = df.drop(columns=["can_mod_post"])

#check if column has been removed correctly
print(df.columns)

False    41309
Name: can_mod_post, dtype: int64
Index(['Unnamed: 0', 'id', 'score', 'created_utc', 'title', 'num_comments',
       'author'],
      dtype='object')


## Clean data

### Derive date and time variables

In [16]:
# derive datetime from unix timestamp
df["created"] = pd.to_datetime(df["created_utc"], unit = 's')

#derive month from datetime
df["created_month"] = pd.DatetimeIndex(df["created"]).month

#derive weekday from datetime
weekday_map = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3:"Thursday", 4: "Friday", 5:"Saturday", 6:"Sunday"}
df["created_weekday"] = pd.DatetimeIndex(df["created"]).weekday

#derive month from datetime
df["created_hour"] = pd.DatetimeIndex(df["created"]).hour

### Title

We need to clean the text data in the title so that we can do some proper text mining. This includes

- Standardising
- Tokenising and removing stop words
- Lemmatising





#### Standardising

We will remove punctuation and clean any other symbols/words as needed.

In [153]:
# make titles lower case
title = df["title"].str.lower()

# replace values within titles

# function to loop through the column and replace substrings
def replace_values(text, dic):
    for x, y in dic.items():
        text = text.str.replace(x, y, regex=True)
    return text

# list of values to be replaced, including punctuation
replace_dict = {"&amp;": "", #xml syntax for &
                "[!\"#$%&()*+,./:;<=>?@[\]^_`{|}~“”-]": "",
                "pnw" : "pacific northwest",
                "  ": " ", #double space
                "’s" :"",  
                "’m" : "",
                "loaves": "loaf"
               }
                
# apply function
title = replace_values(title, replace_dict)

# strip white space at the end
title = title.str.strip()

print(title)

0        first attempt at a starter really hope i manag...
1        skillet dutch oven sourdough in the rainy paci...
2        my last bread of 2019 i used brad and claire r...
3        i started baking in september and i have never...
4                                          sourdough books
                               ...                        
41304                               honey walnut sourdough
41305    so happy with how my sourdough finally turned ...
41306    sourdough bbq faux chicken pizza no 🐔 were har...
41307    got my first starter last week as a christmas ...
41308               last 2020 feeding for dame judi stench
Name: title, Length: 41309, dtype: object


#### Remove stop words

We now remove stop words that don't really help to identify the sentiment or topic of a sentence. 

In [43]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [161]:
# create empty list to store titles without stop words
title_no_stop_words = []

# iterate through each word in each title and append those that are no stop words
# split titles into substrings using space as delimiter
for words in title.str.split(" "):
    x = []
    for word in words:
        if word not in stop_words:
            x.append(word)
    title_no_stop_words.append(x)

# join titles back together
title_no_stop_words = [" ".join(items) for items in title_no_stop_words]

print(title_no_stop_words[:10])

['first attempt starter really hope manage keep alive mixed existence opening second decade', 'skillet dutch oven sourdough rainy pacific northwest', 'last bread 2019 used brad claire recipe alive sourdough starter smelled bit sweet weird bread turned good anyways', 'started baking september never happier decision may new year bring happy starters crackly crusts', 'sourdough books', 'analyzing sourdough', 'ginger tumeric loaf guide decade', 'behold bread majors incite rocky horror sourdough', 'last loaf year', 'wheat flour starter vs rye starter']


#### Lemmatize
Now we want to reduce the inflectional forms of each word into a common base or root. We do this by using a lemmatiser that detects the lemma for each word. 

Read [this article](https://blog.bitext.com/what-is-the-difference-between-stemming-and-lemmatization/) for more information on the difference betweent stemmatising and lemmatising. 

In [190]:
# load model from SpaCy
nlp = en_core_web_sm.load()

# create new list to store lemmatised titles
title_lemmatised = []

# iterate through each word in each title and append the lemmatised version of the word
for words in pd.Series(title_no_stop_words):
    x = []
    for word in nlp(words):
        x.append(word.lemma_)
    title_lemmatised.append(x)

# join titles back together
title_lemmatised = [" ".join(items) for items in title_lemmatised]

['first attempt starter really hope manage keep alive mixed existence open second decade',
 'skillet dutch oven sourdough rainy pacific northwest',
 'last bread 2019 use brad claire recipe alive sourdough starter smell bit sweet weird bread turn good anyways',
 'start bake september never happy decision may new year bring happy starter crackly crust',
 'sourdough book',
 'analyze sourdough',
 'ginger tumeric loaf guide decade',
 'behold bread major incite rocky horror sourdough',
 'last loaf year',
 'wheat flour starter vs rye starter',
 'sourdough starter smell like vomit',
 'achieve number first bake',
 'wheat loaf make family 2 hour 2020 nothing fancy crumb picture comment',
 'hooch',
 'happy turn',
 'third attempt much happy result',
 'new year eve loaf 2nd time ever bake turn pretty good',
 'many sourdough like one mine',
 'happy new ear',
 'turn volume bread singing',
 'look help troubleshoot lack oven spring',
 'maybe dumb question calculate hydration level consider thing like s

# Analysis

## Engagement over time

In [9]:
### Number of submissions by month
df.groupby("created_month")["id"].count()

created_month
1     1584
2     1629
3     3315
4     7569
5     7121
6     4575
7     3633
8     2915
9     2120
10    2205
11    2191
12    2452
Name: id, dtype: int64

In [18]:
### Number of comments by month
df.groupby("created_month")["num_comments"].sum()

created_month
1      9289
2      8081
3     14550
4     31204
5     31009
6     22517
7     20111
8     13030
9     10083
10    11117
11    11967
12    12084
Name: num_comments, dtype: int64

In [12]:
### Number of submissions by weekday
df.groupby("created_weekday")["id"].count()

created_weekday
0    6070
1    5460
2    5576
3    5287
4    5236
5    6262
6    7418
Name: id, dtype: int64

In [26]:
print("Number of submissions:", len(df["id"]))
print("Number of unique users:", len(np.unique(df["author"])))

Number of submissions: 41309
Number of unique users: 16865


In [30]:
### Number of users by month
df.groupby("created_month")["author"].nunique()

created_month
1     1025
2     1025
3     2155
4     4740
5     4293
6     2781
7     2285
8     1848
9     1440
10    1474
11    1457
12    1564
Name: author, dtype: int64

In [66]:
df2 = df[["created_month","author"]]
df2 = df2.groupby('created_month', as_index=True).agg({'author': [pd.Series.nunique, 'count']})
df2.columns = df2.columns.droplevel(level=0)
df2 = df2.reset_index()
df2["submission_per_user"] = df2["count"] / df2["nunique"] 
df2 = df2.rename(columns={"created_month":"month", "nunique":"unique_user", "count": "no_of_submissions"})
df2

,month,unique_user,no_of_submissions,submission_per_user
0,1,1025,1584,1.545366
1,2,1025,1629,1.589268
2,3,2155,3315,1.538283
3,4,4740,7569,1.596835
4,5,4293,7121,1.658747
5,6,2781,4575,1.645092
6,7,2285,3633,1.589934
7,8,1848,2915,1.577381
8,9,1440,2120,1.472222
9,10,1474,2205,1.495929


In [72]:
df2 = df[["created_month", "author", "num_comments"]]

df2 = df2.groupby('created_month', as_index=True).agg({'author': [pd.Series.nunique, 'count'],
                                                                               'num_comments': ['sum']})
df2.columns = df2.columns.droplevel(level=0)
df2 = df2.reset_index()
df2 = df2.rename(columns={"created_month":"month", "nunique":"unique_user", "count": "no_of_submissions", "sum":"total_comments"})
df2["submission_per_user"] = df2["no_of_submissions"] / df2["unique_user"] 
df2["comments_per_submission"] = df2["total_comments"] / df2["no_of_submissions"] 

df2

,month,unique_user,no_of_submissions,total_comments,submission_per_user,comments_per_submission
0,1,1025,1584,9289,1.545366,5.864268
1,2,1025,1629,8081,1.589268,4.960712
2,3,2155,3315,14550,1.538283,4.389140
3,4,4740,7569,31204,1.596835,4.122605
4,5,4293,7121,31009,1.658747,4.354585
5,6,2781,4575,22517,1.645092,4.921749
6,7,2285,3633,20111,1.589934,5.535645
7,8,1848,2915,13030,1.577381,4.469983
8,9,1440,2120,10083,1.472222,4.756132
9,10,1474,2205,11117,1.495929,5.041723


In [14]:
### Number of submissions by hour
df.groupby("created_hour")["id"].count()

created_hour
0     1910
1     1867
2     1761
3     1421
4     1164
5      923
6      730
7      693
8      764
9      853
10     887
11    1065
12    1413
13    1751
14    2153
15    2509
16    2735
17    2689
18    2754
19    2526
20    2454
21    2226
22    2119
23    1942
Name: id, dtype: int64

# Text analysis

- [How To Perform Sentiment Analysis in Python 3 Using the Natural Language Toolkit (NLTK)](https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk)
- [Sentiment Analysis on Reddit News Headlines with Python’s Natural Language Toolkit (NLTK)](https://www.learndatasci.com/tutorials/sentiment-analysis-reddit-headlines-pythons-nltk/)

## Sentiment analysis on title using nltk Vader

In [193]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in titles_lemmatised:
    pol_score = sia.polarity_scores(line)
    pol_score['title'] = line
    results.append(pol_score)

print(results[:3])

[{'neg': 0.0, 'neu': 0.655, 'pos': 0.345, 'compound': 0.6997, 'title': 'first attempt starter really hope manage keep alive mixed existence open second decade'}, {'neg': 0.178, 'neu': 0.822, 'pos': 0.0, 'compound': -0.0772, 'title': 'skillet dutch oven sourdough rainy pacific northwest'}, {'neg': 0.07, 'neu': 0.579, 'pos': 0.351, 'compound': 0.7783, 'title': 'last bread 2019 use brad claire recipe alive sourdough starter smell bit sweet weird bread turn good anyways'}]


In [194]:
new_df = pd.DataFrame.from_records(results)

new_df['label'] = 0
new_df.loc[new_df['compound'] > 0.2, 'label'] = 1
new_df.loc[new_df['compound'] < -0.2, 'label'] = -1
new_df.head()

,neg,neu,pos,compound,title,label
0,0.000,0.655,0.345,0.6997,first attempt starter really hope manage keep ...,1
1,0.178,0.822,0.000,-0.0772,skillet dutch oven sourdough rainy pacific nor...,0
2,0.070,0.579,0.351,0.7783,last bread 2019 use brad claire recipe alive s...,1
3,0.333,0.667,0.000,-0.7181,start bake september never happy decision may ...,-1
4,0.000,1.000,0.000,0.0000,sourdough book,0


In [195]:
df2 = new_df[['title', 'label']]

print(df2["label"].value_counts())
print(df2["label"].value_counts(normalize=True) * 100)

 0    20029
 1    17308
-1     3972
Name: label, dtype: int64
 0    48.485802
 1    41.898860
-1     9.615338
Name: label, dtype: float64


In [75]:
pd.DataFrame(df2.loc[df2["label"]== -1, "title"]).to_csv(".//data/processed/negative_title.csv")

In [196]:
df2.loc[df2["label"]== 1]

,title,label
0,first attempt starter really hope manage keep ...,1
2,last bread 2019 use brad claire recipe alive s...,1
10,sourdough starter smell like vomit,1
14,happy turn,1
15,third attempt much happy result,1
...,...,...
41299,right mr demille -PRON- be ready closeup,1
41301,happy new year,1
41302,lot fun improve score skill 😊 😊,1
41305,happy sourdough finally turn month dense pale ...,1


#### most frequent words

In [147]:
from nltk.tokenize import word_tokenize, RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(example)

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def process_text(titles):
    tokens = []
    for line in titles:
        toks = tokenizer.tokenize(line)
        toks = [t.lower() for t in toks if t.lower() not in stop_words]
        tokens.extend(toks)
    
    return tokens

In [197]:
title_tokens = process_text(df2["title"])
word_freq = nltk.FreqDist(title_tokens)

word_freq.most_common(100)

[('sourdough', 12047),
 ('loaf', 8432),
 ('first', 7994),
 ('starter', 6434),
 ('bread', 4522),
 ('make', 4337),
 ('get', 3781),
 ('pron', 2926),
 ('bake', 2909),
 ('flour', 2749),
 ('good', 2696),
 ('attempt', 2620),
 ('time', 2606),
 ('crumb', 2518),
 ('oven', 2445),
 ('use', 2080),
 ('recipe', 1946),
 ('hydration', 1895),
 ('finally', 1857),
 ('try', 1825),
 ('one', 1739),
 ('day', 1698),
 ('happy', 1675),
 ('dough', 1642),
 ('well', 1632),
 ('help', 1568),
 ('like', 1494),
 ('go', 1449),
 ('look', 1386),
 ('rise', 1373),
 ('today', 1347),
 ('wheat', 1343),
 ('whole', 1282),
 ('ever', 1227),
 ('second', 1215),
 ('ear', 1184),
 ('still', 1179),
 ('think', 1156),
 ('start', 1140),
 ('rye', 1125),
 ('discard', 1085),
 ('score', 1021),
 ('turn', 1012),
 ('new', 985),
 ('pretty', 959),
 ('week', 904),
 ('little', 883),
 ('need', 877),
 ('dutch', 847),
 ('yet', 822),
 ('pizza', 819),
 ('hour', 818),
 ('much', 818),
 ('taste', 818),
 ('spring', 813),
 ('work', 811),
 ('tip', 807),
 ('great

#### most frequent positive words

In [198]:
pos_lines = list(df2.loc[df2["label"] == 1, "title"])

pos_tokens = process_text(pos_lines)
pos_freq = nltk.FreqDist(pos_tokens)

pos_freq.most_common(20)

[('sourdough', 4966),
 ('loaf', 4563),
 ('first', 3974),
 ('starter', 2639),
 ('good', 2452),
 ('make', 2278),
 ('bread', 2145),
 ('get', 2074),
 ('pron', 1956),
 ('happy', 1585),
 ('crumb', 1477),
 ('well', 1461),
 ('bake', 1435),
 ('time', 1428),
 ('help', 1359),
 ('attempt', 1347),
 ('like', 1292),
 ('flour', 1249),
 ('oven', 1244),
 ('one', 1153)]

#### most frequent negative words

In [151]:
neg_lines = list(df2.loc[df2["label"] == -1, "title"])

neg_tokens = process_text(neg_lines)
neg_freq = nltk.FreqDist(neg_tokens)

neg_freq.most_common(20)

[('sourdough', 933),
 ('starter', 807),
 ('discard', 608),
 ('first', 583),
 ('loaf', 530),
 ('bread', 370),
 ('wrong', 342),
 ('bad', 240),
 ('oven', 234),
 ('made', 212),
 ('time', 194),
 ('dough', 191),
 ('flour', 189),
 ('attempt', 158),
 ('crumb', 148),
 ('recipe', 143),
 ('hydration', 132),
 ('finally', 123),
 ('day', 118),
 ('got', 117)]

In [199]:
neg_lines = list(df2.loc[df2["label"] == -1, "title"])

neg_tokens = process_text(neg_lines)
neg_freq = nltk.FreqDist(neg_tokens)

neg_freq.most_common(20)

[('sourdough', 980),
 ('starter', 835),
 ('first', 634),
 ('discard', 616),
 ('loaf', 588),
 ('make', 419),
 ('bread', 410),
 ('get', 359),
 ('wrong', 341),
 ('go', 282),
 ('pron', 272),
 ('bad', 254),
 ('bake', 244),
 ('oven', 242),
 ('use', 233),
 ('time', 232),
 ('crumb', 221),
 ('attempt', 210),
 ('dough', 210),
 ('flour', 209)]

In [200]:
df2.loc[df2["label"] == 1, "title"]

0        first attempt starter really hope manage keep ...
2        last bread 2019 use brad claire recipe alive s...
10                      sourdough starter smell like vomit
14                                              happy turn
15                         third attempt much happy result
                               ...                        
41299             right mr demille -PRON- be ready closeup
41301                                       happy new year
41302                      lot fun improve score skill 😊 😊
41305    happy sourdough finally turn month dense pale ...
41307    get first starter last week christmas present ...
Name: title, Length: 17308, dtype: object

## Topic analysis